Copyright (C) 2023 Sean Ming-Fong Sie <seansie07@gmail.com>

This work is licensed under the Creative Commons Attribution-NonCommercial
4.0 International License. To view a copy of this license, visit
http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

In [13]:
from pyqubo import Spin
# set S = {4, 2, 7, 1}
s1, s2, s3, s4 = Spin("s1"), Spin("s2"), Spin("s3"), Spin("s4")
# Quadratic Unconstrained Binary Optimization（QUBO）for **2 as Hamiltonian 
H = (4*s1 + 2*s2 + 7*s3 + s4)**2
print("QUBO Hamiltonian:",H)
# Compile Quantum Computer or Classical Binary Optimization Solver as Hamiltonian to a model
model = H.compile()
print("Model:", model)
# Convert QUBO to Ising Hamiltonian, and get the offset as constant
qubo, offset = model.to_qubo()
print("QUBO Result:",qubo)
print("Offset:", offset)

QUBO Hamiltonian: ((Spin('s4') + (7.000000 * Spin('s3')) + (4.000000 * Spin('s1')) + (2.000000 * Spin('s2'))) * (Spin('s4') + (7.000000 * Spin('s3')) + (4.000000 * Spin('s1')) + (2.000000 * Spin('s2'))))
Model: <cpp_pyqubo.Model object at 0x7fa8d0a6cd30>
QUBO Result: {('s1', 's1'): -160.0, ('s4', 's2'): 16.0, ('s3', 's1'): 224.0, ('s2', 's2'): -96.0, ('s4', 's1'): 32.0, ('s1', 's2'): 64.0, ('s3', 's2'): 112.0, ('s3', 's3'): -196.0, ('s4', 's4'): -52.0, ('s4', 's3'): 56.0}
Offset: 196.0


In [17]:
import neal
sampler = neal.SimulatedAnnealingSampler() # Simulated Annealing Sampler using D-Wave
print("Sampler:", sampler)
bqm = model.to_bqm() # Convert QUBO to Binary Quadratic Model
print("Binary Quadratic Model:", bqm)
sampleset = sampler.sample(bqm, num_reads=10) # Retrieve 10 samples
print("Sample Set:", sampleset)
decoded_samples = model.decode_sampleset(sampleset) # Decode the sampleset
print("Decoded Sample:", decoded_samples)
best_sample = min(decoded_samples, key=lambda x: x.energy) # Retrieve the best sample
print("Best Sample:", best_sample)
best_sample.sample # doctest: +SKIP, Retrieve the best sample's value
print("Best Sample:", best_sample.sample)

Sampler: <dwave.samplers.sa.sampler.SimulatedAnnealingSampler object at 0x7fa8d0a820a0>
Binary Quadratic Model: BinaryQuadraticModel({'s4': -52.0, 's2': -96.0, 's3': -196.0, 's1': -160.0}, {('s2', 's4'): 16.0, ('s3', 's4'): 56.0, ('s3', 's2'): 112.0, ('s1', 's4'): 32.0, ('s1', 's2'): 64.0, ('s1', 's3'): 224.0}, 196.0, 'BINARY')
Sample Set:   s1 s2 s3 s4 energy num_oc.
0  0  0  1  0    0.0       1
1  1  1  0  1    0.0       1
2  1  1  0  1    0.0       1
3  1  1  0  1    0.0       1
4  1  1  0  1    0.0       1
5  0  0  1  0    0.0       1
6  0  0  1  0    0.0       1
7  0  0  1  0    0.0       1
8  1  1  0  1    0.0       1
9  0  0  1  0    0.0       1
['BINARY', 10 rows, 10 samples, 4 variables]
Decoded Sample: [DecodedSolution({s3:1, s4:0, s2:0, s1:0}, energy=0.000000), DecodedSolution({s1:0, s2:0, s4:0, s3:1}, energy=0.000000), DecodedSolution({s3:1, s4:0, s2:0, s1:0}, energy=0.000000), DecodedSolution({s3:1, s4:0, s2:0, s1:0}, energy=0.000000), DecodedSolution({s1:0, s2:0, s4:0, s3

In [45]:
import pandas as pd
from pyqubo import Binary

# 指定CSV文件的路径
file_path = "../data_p/data.address.csv"

# 使用pandas的read_csv函數讀取CSV文件
df = pd.read_csv(file_path)
print(df)

# 獲取DataFrame中所有列名，並存儲為列表
all_columns = df.columns.tolist()

# 選擇要用於構建Set的列名
selected_columns = all_columns[1:10] + ["class"]  #選擇特定列名範圍，例如從第1列到第9列

# 從DataFrame中選擇所需的列，然後將數據轉換為字典
qubo_set = df[selected_columns].to_dict(orient='list')

# 從生成的 Set 數據創建 QUBO
qubo_model = 0  # 初始化 QUBO 模型

# 添加 QUBO 项到模型中
for key, value in qubo_set.items():
    qubo_model += value[0] * Binary(key) ** 2

# 將 "class" 列也加入 QUBO 模型中
qubo_model += qubo_set["class"][0] * Binary("class") ** 2

# 顯示生成的 QUBO Hamiltonian
print("QUBO Hamiltonian:", qubo_model)

# 編譯為模型
compiled_model = qubo_model.compile()
print("Model:", compiled_model)

# 將 QUBO 轉換為 Ising Hamiltonian，並獲得偏移量作為常數
qubo, offset = compiled_model.to_qubo()
print("QUBO Result:", qubo)
print("Offset:", offset)


    n_tx  total_days  total_spent_btc  total_received_btc  total_spent_usd  \
0    1.0         1.0         0.012863            0.012863         9.529224   
1    1.0         1.0         0.000275            0.000275         0.203728   
2    3.0         1.0         0.000300            0.000300         0.206610   
3    1.0         1.0         0.000100            0.000100         0.068870   
4    2.0         1.0         2.406666            2.406666      1657.471179   
5    1.0         1.0         0.169892            0.169892       117.004829   
6    1.0         1.0         0.107520            0.107520        74.048791   
7    3.0         1.0         0.009000            0.009000         6.198300   
8    1.0         1.0         0.005500            0.005500         4.074559   
9    7.0         1.0         0.005718            0.005718         3.991833   
10   2.0         1.0         0.029707            0.029707        20.459466   
11   1.0         1.0         0.066994            0.066994       

In [44]:
import neal
from pyqubo import Binary

# 编译为模型
compiled_model = qubo_model.compile()
print("Model:", compiled_model)

# 将 QUBO 转换为 Binary Quadratic Model
bqm = compiled_model.to_bqm()

# 使用模拟器进行采样
sampler = neal.SimulatedAnnealingSampler()
print("Sampler:", sampler)

# 采样得到样本
num_reads = 10
sampleset = sampler.sample(bqm, num_reads=num_reads)
print("Sample Set:", sampleset)

# 解码采样结果
decoded_samples = compiled_model.decode_sampleset(sampleset)
print("Decoded Sample:", decoded_samples)

# 寻找最佳样本
best_sample = min(decoded_samples, key=lambda x: x.energy)
print("Best Sample:", best_sample)

# 获取最佳样本的值
best_sample_value = best_sample.sample
print("Best Sample:", best_sample_value)


Model: <cpp_pyqubo.Model object at 0x7fa8e0b05230>
Sampler: <dwave.samplers.sa.sampler.SimulatedAnnealingSampler object at 0x7fa8e0b1e4f0>
Sample Set:   class mean_balance_btc mean_balance_usd ... total_spent_usd energy num_oc.
0     0                0                0 ...               0    0.0       1
1     0                0                0 ...               0    0.0       1
2     0                0                0 ...               0    0.0       1
3     0                0                0 ...               0    0.0       1
4     1                0                0 ...               0    0.0       1
5     0                0                0 ...               0    0.0       1
6     0                0                0 ...               0    0.0       1
7     0                0                0 ...               0    0.0       1
8     0                0                0 ...               0    0.0       1
9     1                0                0 ...               0    0.0       1
['